#  Informe de consumo 

Tras obtener todas las metricas  del mejor modelo es hora de presentar un resumen del los resultados obtenidos
con el mejor algoritmo entrenado con un tiempo determinado y con un diferente metodo de interpolacion.

Destacar que todos los resultados de las prediciones los salvamos en la ruta : C:/Users/USUARIO/jupyter_notebooks/segundo_experimento_datos_acoplados_temporalmente-main/data/NILM/Results

Asimismo los resultados ademas de presentarlos en este cuaderno lo salvamos en:
c:/Users/USUAIO/jupyter_notebooks/segundo_experimento_datos_acoplados_temporalmente-main/data/results.htm


En primer lugar, debemos cargar las librerías como plotly, nilmtk,pandas, datetime y codecs.

In [10]:
import pandas as pd
import datetime
import codecs
from plotly.io import to_html
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px 
from plotly.subplots import make_subplots
from nilmtk import DataSet
import warnings
from IPython.display import SVG

warnings. filterwarnings("ignore")
plt.style.use('bmh')


## Breve explicacion de la función de informe
Creamos una función que recupera las prediciones del mejor modelo y sobre esta calcula el total de energia  y potencia consumida. 

Ademas ofrece otros datos interesantes como son el periodo de inicio y fin ,la tasa de muestreo , etc.

Asimismo tambien sacamos la grafica del consumo desagregado de los difrenetes aplicativos frente a a la lectura del contador principal.

Como relevante, sacamos toda la información interesante,tambien en un fichero en formato  html, que se ira sobreescribiendo a medida que ejecutemos este cuaderno. 

In [11]:
def report():
        
    #Tomamos la mejor prediccion, que para este segundo experimento de datos sincronizados temporalmente sin tomar los 
    #transitorios es la siguinte combinación:
    # -algoritmo FHMM  
    # -el mejor tiempo de muestreo(60seg) 
    # -el mejor metodo de interpolacion (metodo first)
    
    results = DataSet('./data/NILM/Results/FHMM_60s_first_pred.h5')   
    #ruta del fichero de salida 
    html_dir =('./data/results.htm')   

    
    labels = ['Main', 'Electric furnace', 'Microwave','Television','Kettle','Vacuum cleaner','Electric space heater','Electric shower heater','Fan','Fridge','Freezer'] #etiquetas
    
    results_elec = results.buildings[1].elec
    
    print (results_elec) #pintamos los diferentes aplicativos con sus asociaciones
   
   
    #pie
    df1=pd.DataFrame(results_elec.submeters().fraction_per_meter(), columns=['Fracción'])
    df1['Energía']=round(df1['Fracción']*results_elec.total_energy()[0],1)

    
    fig1 = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
    fig1.add_trace(go.Pie(values=df1['Fracción'], name='Porcentaje', hoverinfo="label+percent+name",
                          textinfo='label+percent', labels=labels, sort=False), 1, 1)
    fig1.add_trace(go.Pie(values=df1['Energía'], name='Energía', hoverinfo="label+value+name", textinfo='label+value',
                          labels=labels, sort=False, texttemplate="%{value:.1f} kWh"), 1, 2)
    fig1.update_traces(hole=.3)
    fig1.update_layout(width=900, height=450,
         annotations=[dict(text='%', x=0.21, y=0.5, font_size=22, showarrow=False),
                      dict(text='kWh', x=0.81, y=0.5, font_size=22, showarrow=False)])
    
    fig1.show() #pintamos el resultado por pantalla
    pie=fig1.to_html()# convertimos el grafico en un objeto para poderlo copiar en un fichero html
   
    
    #lines
    df2=results_elec.select(instance=[2,3,4,5,6,7,8,9,10,11]).dataframe_of_meters(ac_type='active')
    df2.columns=labels[:10] #old 5
    fig2=px.line(df2, width=900, height=500,labels={'variable':'Dispositivos', 'index':'Fecha', 'value':'Potencia (W)'})
  
    fig2.show() #pintamos el resultado por pantalla
    lines=fig2.to_html() # convertimos el grafico en un objeto para poderlo copiar en un fichero html
   
    
    #metadata
    date=results.metadata['date'] #recuperamos del dataset la fecha
    date=datetime.datetime.strptime(date,'%Y-%m-%dT%H:%M:%S').strftime('%d/%m/%Y %H:%M:%S')
    
    sample_period=results.metadata['meter_devices']['mains']['sample_period'] #recuperamos del dataset el tiempo de muestreo
    sample_period =str (sample_period)+ 'segundos'
    print ('Sample_period:',sample_period)
    
    start=results.metadata['timeframe']['start'] #recuperamos del dataset la hora de inicio
    start=datetime.datetime.strptime(start,'%Y-%m-%dT%H:%M:%S+01:00').strftime('%d/%m/%Y %H:%M:%S')
    print ('Start:',start)
    
    end=results.metadata['timeframe']['end'] #recuperamos del dataset la hora de finalizacion
    end=datetime.datetime.strptime(end,'%Y-%m-%dT%H:%M:%S+01:00').strftime('%d/%m/%Y %H:%M:%S')
    print ('End:',end)


    
    
    #a continuacion creamos una cadena en formato HTML estatico que incluye 
    #los graficos y los datos mas relevantes recuperados del dataset    
    
    msg ="""
    <html>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="stylesheet" href="https://www.w3schools.com/w3css/4/w3.css">
    <head>  <meta http-equiv="content-type" content="text/html; charset=UTF-8"> </head>
       <body>
        <h1><td><img src="logotipo.png" width="160" ></td><b>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Desagregación Consumo energético con OZM y NILMTK</b></h1>
          <br>  
        <h4><b>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Localización y fecha</b></h4>
      
        <table >
          <tbody>
            <tr>
            <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
             <td>Local:</td> <td>Laboratorio Electrotecnia Escuela Politecnica </td> </tr>
             <tr>
             <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
                        <td >Fecha:          </td>     <td >"""+date+""" </td>
            </tr>
          </tbody>
        </table>
        <h4><b>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Energía Consumida</b></h4>
        <table >
          <tbody>
            <tr>
            <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
              <td >Fecha
                Inicio:</td>
              <td >"""+start+"""<br>
                </td>
            </tr>
            <tr>
            <td></td>
              <td >Fecha Fin:</td>
              <td >"""+end+"""<br>
                </td>
            </tr>
            <tr>
            <td></td>
              <td >Periodo de
                muestreo:</td>
              <td >"""+sample_period+"""<br>
              </td>
            </tr>
          </tbody>
        </table>
        <p></p>
        <h4><b>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Gráfico de energía desagregada</b></h4>
        """+pie+"""
        <h4><b>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Gráfico de potencia instantánea desagregada</b> </h4>
        """+lines+"""
        <p><br>
        </p>
        <p>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Se puede encontrar mas información en  <a href="https://github.com/crn565/segundo_experimento_datos_acoplados_temporalmente">github.com/crn565
                  </a></p>
       &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Octubre 2022 by CRN<span><br>
        </span><br>
        <br>
      </body>
     </html>"""

    with codecs.open(html_dir, 'w', encoding='utf8') as f:
        f.write(msg)     #escribimos la cadena msg en el fichero de salida
        f.close()        #cerramos el fichero
    return print('!El informe  ya se ha creado!')
    
    

In [12]:
#llamada a la función
report()

MeterGroup(meters=
  ElecMeter(instance=1, building=1, dataset='FHMM', site_meter, appliances=[])
  ElecMeter(instance=2, building=1, dataset='FHMM', appliances=[Appliance(type='electric furnace', instance=1)])
  ElecMeter(instance=3, building=1, dataset='FHMM', appliances=[Appliance(type='microwave', instance=1)])
  ElecMeter(instance=4, building=1, dataset='FHMM', appliances=[Appliance(type='television', instance=1)])
  ElecMeter(instance=5, building=1, dataset='FHMM', appliances=[Appliance(type='kettle', instance=1)])
  ElecMeter(instance=6, building=1, dataset='FHMM', appliances=[Appliance(type='vacuum cleaner', instance=1)])
  ElecMeter(instance=7, building=1, dataset='FHMM', appliances=[Appliance(type='electric space heater', instance=1)])
  ElecMeter(instance=8, building=1, dataset='FHMM', appliances=[Appliance(type='electric shower heater', instance=1)])
  ElecMeter(instance=9, building=1, dataset='FHMM', appliances=[Appliance(type='fan', instance=1)])
  ElecMeter(instance=10, 

Sample_period: 60segundos
Start: 07/03/2023 12:10:00
End: 07/03/2023 12:41:00
!El informe  ya se ha creado!
